In [15]:
import requests
import grab
import json
import logging
import codecs
import numpy as np
import re
from lxml.etree import ElementTree, fromstring
from bs4 import BeautifulSoup
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool
from IPython.display import HTML

Основная ссылка на отели Майорки. Параметры поиска: два человека, одна комната, без детей, даты: 06.04.2018 - 08.04.2018

In [ ]:
url3 = 'https://www.booking.com/searchresults.ru.html?aid=304142&label=gen173nr-1DCAEoggJCAlhY' + \
    'SDNYBGjCAYgBAZgBIcIBA3gxMcgBDNgBA-gBAZICAXmoAgM&sid=37ffdc5e5a0795086e5bcb2fafb379d8&checkin_m' + \
    'onth=4&checkin_monthday=6&checkin_year=2018&checkout_month=4&checkout_monthday=8&checkout_year' + \
    '=2018&class_interval=1&fe_new_tab=1&group_adults=2&group_children=0&label_click=undef&lsf=oos%' + \
    '7C1%7C-1&nflt=oos%3D1%3B&no_rooms=1&region=767&room1=A%2CA&sb_price_type=total&src=searchresul' + \
    'ts&ss=%D0%9C%D0%B0%D0%B9%D0%BE%D1%80%D0%BA%D0%B0&ssb=empty&ssne=%D0%9C%D0%B0%D0%B9%D0%BE%D1%80' + \
    '%D0%BA%D0%B0&ssne_untouched=%D0%9C%D0%B0%D0%B9%D0%BE%D1%80%D0%BA%D0%B0&rows=15&offset=0#sort_by'

Функция, чтобы получать URL поиска по заданным датам и на конкретную страницу поиска. Плюс в запросе выставлена галочка на "Доступные предложение" -- это значит места есть

In [10]:
def set_dates_and_page_num(checkin, checkout, page_num):
    '''
    Creates an URL with given checkin and checkout dates
    params: checkin date = (d,m,y)
           checkout date = (d,m,y)
    returns: the URL to first page of search results to these dates
    '''
    url = 'https://www.booking.com/searchresults.ru.html?aid=304142&label=gen173nr-1DCAEoggJCAlhY' + \
    'SDNYBGjCAYgBAZgBIcIBA3gxMcgBDNgBA-gBAZICAXmoAgM&sid=37ffdc5e5a0795086e5bcb2fafb379d8&checkin_m' + \
    'onth={2}&checkin_monthday={1}&checkin_year={3}&checkout_month={5}&checkout_monthday={4}&checkout_year' + \
    '={6}&class_interval=1&fe_new_tab=1&group_adults=2&group_children=0&label_click=undef&lsf=oos%' + \
    '7C1%7C-1&nflt=oos%3D1%3B&no_rooms=1&region=767&room1=A%2CA&sb_price_type=total&src=searchresul' + \
    'ts&ss=%D0%9C%D0%B0%D0%B9%D0%BE%D1%80%D0%BA%D0%B0&ssb=empty&ssne=%D0%9C%D0%B0%D0%B9%D0%BE%D1%80' + \
    '%D0%BA%D0%B0&ssne_untouched=%D0%9C%D0%B0%D0%B9%D0%BE%D1%80%D0%BA%D0%B0&rows=15&offset={0}#sort_by'
    return url.format(page_num*15, *(checkin+checkout))


In [13]:
tmp = set_dates_and_page_num((1,4,2018,),(4,4,2018,),3)
print(tmp)

https://www.booking.com/searchresults.ru.html?aid=304142&label=gen173nr-1DCAEoggJCAlhYSDNYBGjCAYgBAZgBIcIBA3gxMcgBDNgBA-gBAZICAXmoAgM&sid=37ffdc5e5a0795086e5bcb2fafb379d8&checkin_month=4&checkin_monthday=1&checkin_year=2018&checkout_month=4&checkout_monthday=4&checkout_year=2018&class_interval=1&fe_new_tab=1&group_adults=2&group_children=0&label_click=undef&lsf=oos%7C1%7C-1&nflt=oos%3D1%3B&no_rooms=1&region=767&room1=A%2CA&sb_price_type=total&src=searchresults&ss=%D0%9C%D0%B0%D0%B9%D0%BE%D1%80%D0%BA%D0%B0&ssb=empty&ssne=%D0%9C%D0%B0%D0%B9%D0%BE%D1%80%D0%BA%D0%B0&ssne_untouched=%D0%9C%D0%B0%D0%B9%D0%BE%D1%80%D0%BA%D0%B0&rows=15&offset=45#sort_by


Можно посмотреть, что содержится в HTML этой ссылки

In [16]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '+ 
    'Ubuntu Chromium/62.0.3202.75 Chrome/62.0.3202.75 Safari/537.36',
}
g = grab.Grab(headers=headers)

cookies = {'bkng':"11UmFuZG9tSVYkc2RlIyh9Yaa29%2F3xUOLbA1O770KQL%2Bz6eiWwZ2kqlb%2Bvw%2FwM9Gr" + \
        "XBWhhmhTCVHab7ct1N0lsqxgqlGvH5BBioyJm%2FcjYpww8NiDnm%2FGQaCWvfArsNeAPAY%2BzjOaY" + \
        "VTabpbR67AnSIUDh58I%2B40hZQRzSrDmFUzj5rmw4sW4%2FQ5JrXroJ7fmZd6nn9cI6gUmF340mRwh" + \
        "F1cATGzE0Cw46Ciqgh4%2F90wabGE9Q5j3xsQ%3D%3D", \
        'lastSeen':'1511865744489'}

g.setup(cookies=cookies)
g.go(tmp)
html = g.response.unicode_body()
soup = BeautifulSoup(html, "lxml")
print(soup)

<!DOCTYPE html>
<!--
You know you could be getting paid to poke around in our code?
We're hiring designers and developers to work in Amsterdam:
http://www.workingatbooking.com/
--><!-- wdot-802 --><html class="noJS b_chrome b_chrome_62 supports_inline-block supports_flexbox_unprefixed supports_fontface " lang="ru" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# booking_com: http://ogp.me/ns/fb/booking_com#">
<head profile="http://a9.com/-/spec/opensearch/1.1/">
<meta content="origin-when-cross-origin" name="referrer"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<script>
;(function(w){
var ts = +new Date();
w.PageLoadTimer = {};
w.PageLoadTimer.start = ts;
}(window));
</script>
<link data-main-css="1" href="https://t-ec.bstatic.com/static/css/main_ecv6/2251749e09c900948aafdb5070ffa5502064ebb5.css" rel="stylesheet"/>
<link href="https://s-ec.bstatic.com/static/css/main_exps_ecv6/06c899a883f40e1e70d09b688f33e0e19cc93c32.css" rel="stylesheet"/>
<link href="

Получим ссылки на все отели с 10 первых страниц, доступные на эту дату

In [20]:
def get_hotels(g):
    
    def hotel_name_link(class_):
        return class_ and re.compile('hotel_name_link').search(class_)

    html = g.response.unicode_body()
    soup = BeautifulSoup(html, "lxml")
    hotels = soup.find_all("a", class_=hotel_name_link)
    urls = []
    for hotel in hotels:
        urls.append(hotel['href'])
    return

In [21]:
checkin = (1,4,2018,)
checkout = (4,4,2018,)
N = 10

all_pages = [set_dates_and_page_num(checkin, checkout, x) for x in range(N)]

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '+ 
    'Ubuntu Chromium/62.0.3202.75 Chrome/62.0.3202.75 Safari/537.36',
}
g = grab.Grab(headers=headers)

cookies = {'bkng':"11UmFuZG9tSVYkc2RlIyh9Yaa29%2F3xUOLbA1O770KQL%2Bz6eiWwZ2kqlb%2Bvw%2FwM9Gr" + \
        "XBWhhmhTCVHab7ct1N0lsqxgqlGvH5BBioyJm%2FcjYpww8NiDnm%2FGQaCWvfArsNeAPAY%2BzjOaY" + \
        "VTabpbR67AnSIUDh58I%2B40hZQRzSrDmFUzj5rmw4sW4%2FQ5JrXroJ7fmZd6nn9cI6gUmF340mRwh" + \
        "F1cATGzE0Cw46Ciqgh4%2F90wabGE9Q5j3xsQ%3D%3D", \
        'lastSeen':'1511865744489'}

g.setup(cookies=cookies)
hotels_url = []
for url in all_pages:
    try:
        g.go(url)
        hotels_url.append(get_hotels(g))

    except Exception as e:
        logging.error(str(e))
        logging.error('Error on ')
